# Project Title
### Data Engineering Capstone Project

#### Project Summary
For this project, I pulled the data from the AWS S3 and store them into RedShift database through a star schema.  This schema is designed to answer the question on where the immigrants were coming from and where do they go in US.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import configparser

In [38]:
import re
from collections import defaultdict
from datetime import datetime, timedelta
import psycopg2
from sqlalchemy import create_engine






In [3]:
#I want to display all 28 columns in the notebook
pd.set_option('display.max_columns', 28)

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>
I used the immigration and the city demographics dataset provided by Udacity for this project.  
Once the data's read into the pandas dataframe, I then push it to Postgres.  


#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

The immigration dataset is provided by US National Tourism and Trade Office.  And the city demographic dataset is coming from OpenSoft.

The immigration dataset contains the entry logs into the US borders.

The city demographic dataset contains information about the demographic in the US cities.

In [4]:
# Read in the data here

IMMIGRATION_CHUNK_SIZE = 500000
IMMIGRATION_DATA_FILES = [
    '../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat',
    '../../data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat',
    '../../data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat',
    '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat',
    '../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat',
    '../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat',
    '../../data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat',
    '../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat',
    '../../data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat',
    '../../data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat',
    '../../data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat',
    '../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat'
]

HEADER_FILE = 'I94_SAS_Labels_Descriptions.SAS'


Loading one immigration file to explore the data

In [5]:
filename = IMMIGRATION_DATA_FILES[0]
iterator = pd.read_sas(
    filename, 'sas7bdat', encoding="ISO-8859-1", chunksize=IMMIGRATION_CHUNK_SIZE
)
df = next(iterator)



In [6]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,7.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,20.0,3.0,1.0,NaN,NaN,NaN,T,NaN,NaN,NaN,1996.0,D/S,M,NaN,LH,346608285.0,424,F1
1,8.0,2016.0,1.0,101.0,101.0,BOS,20465.0,1.0,MA,NaN,20.0,3.0,1.0,NaN,NaN,NaN,T,NaN,NaN,NaN,1996.0,D/S,M,NaN,LH,346627585.0,424,F1
2,9.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20480.0,17.0,2.0,1.0,NaN,NaN,NaN,T,N,NaN,M,1999.0,07152016,F,NaN,AF,381092385.0,338,B2
3,10.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,45.0,2.0,1.0,NaN,NaN,NaN,T,N,NaN,M,1971.0,07152016,F,NaN,AF,381087885.0,338,B2
4,11.0,2016.0,1.0,101.0,101.0,BOS,20469.0,1.0,CT,20499.0,12.0,2.0,1.0,NaN,NaN,NaN,T,N,NaN,M,2004.0,07152016,M,NaN,AF,381078685.0,338,B2


In [7]:
#Verify if cicid is the primary key
df[['cicid']].groupby(['cicid']).sum().reset_index()

,cicid
0,7.0
1,8.0
2,9.0
3,10.0
4,11.0
5,12.0
6,15.0
7,17.0
8,18.0
9,20.0


In [8]:
city_df = pd.read_csv('us-cities-demographics.csv', delimiter=';')
print(city_df.shape)
city_df.head()

(2891, 12)


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.
Upon inspecting the immigration data, few issues were identified.

1. Bad birth year under the i94bir column.  Since the report's made in 2016, any birthday year after 2016 will be replaced to NULL.  And any year outside of the 1900 - 2006 will also be replaced with NULL value.

2. Since the data's divided into multiple files, we need to ensure the columns from each file are consistent.


In [9]:
#put small dataframe from each month into a list
small_list = []
for fname in IMMIGRATION_DATA_FILES:
    small_iterator = pd.read_sas(fname, 'sas7bdat', encoding='ISO-8859-1', chunksize=20)
    small_list.append(next(small_iterator))

#create a dictionary map for each month to list of the columns
column_names_by_files = {t[0].split('/')[-1].split('_')[1][:3]: list(t[1].columns.values)
                        for t in zip(IMMIGRATION_DATA_FILES, small_list)
                        }

In [10]:
column_names_by_files

{'jan': ['cicid',
  'i94yr',
  'i94mon',
  'i94cit',
  'i94res',
  'i94port',
  'arrdate',
  'i94mode',
  'i94addr',
  'depdate',
  'i94bir',
  'i94visa',
  'count',
  'dtadfile',
  'visapost',
  'occup',
  'entdepa',
  'entdepd',
  'entdepu',
  'matflag',
  'biryear',
  'dtaddto',
  'gender',
  'insnum',
  'airline',
  'admnum',
  'fltno',
  'visatype'],
 'feb': ['cicid',
  'i94yr',
  'i94mon',
  'i94cit',
  'i94res',
  'i94port',
  'arrdate',
  'i94mode',
  'i94addr',
  'depdate',
  'i94bir',
  'i94visa',
  'count',
  'dtadfile',
  'visapost',
  'occup',
  'entdepa',
  'entdepd',
  'entdepu',
  'matflag',
  'biryear',
  'dtaddto',
  'gender',
  'insnum',
  'airline',
  'admnum',
  'fltno',
  'visatype'],
 'mar': ['cicid',
  'i94yr',
  'i94mon',
  'i94cit',
  'i94res',
  'i94port',
  'arrdate',
  'i94mode',
  'i94addr',
  'depdate',
  'i94bir',
  'i94visa',
  'count',
  'dtadfile',
  'visapost',
  'occup',
  'entdepa',
  'entdepd',
  'entdepu',
  'matflag',
  'biryear',
  'dtaddto',
 

In [11]:
#list of column names as key and the values are list of the month
column_names_by_files_reversed = defaultdict(list)
for k, v in column_names_by_files.items():
    column_names_by_files_reversed[','.join(v)].append(k)
print(len(column_names_by_files_reversed))
column_names_by_files_reversed.values()

2


dict_values([['jan', 'feb', 'mar', 'apr', 'may', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'], ['jun']])

looks like June has some problem.  More investigation..

In [12]:
normal_months = list(small_list[0].columns.values)
problem_june = list(small_list[5].columns.values)
print('{} vs {}'.format(len(normal_months), len(problem_june)))
for zipped in (zip(normal_months, problem_june)):
    print(zipped)

28 vs 34
('cicid', 'cicid')
('i94yr', 'i94yr')
('i94mon', 'i94mon')
('i94cit', 'i94cit')
('i94res', 'i94res')
('i94port', 'i94port')
('arrdate', 'arrdate')
('i94mode', 'i94mode')
('i94addr', 'i94addr')
('depdate', 'depdate')
('i94bir', 'i94bir')
('i94visa', 'i94visa')
('count', 'count')
('dtadfile', 'validres')
('visapost', 'delete_days')
('occup', 'delete_mexl')
('entdepa', 'delete_dup')
('entdepd', 'delete_visa')
('entdepu', 'delete_recdup')
('matflag', 'dtadfile')
('biryear', 'visapost')
('dtaddto', 'occup')
('gender', 'entdepa')
('insnum', 'entdepd')
('airline', 'entdepu')
('admnum', 'matflag')
('fltno', 'biryear')
('visatype', 'dtaddto')


In [13]:
normal_months == [correct_column for correct_column in problem_june if not (correct_column.startswith('delete_') or correct_column =='validres')]



True

Looks like for some reason, the month of june has additional columns that was not prepared correctly.  We just need to make sure we exclude them at the time of insert.


In [14]:
#### Exploring the city demographic data
#Looking at one city sample:
city_df[city_df['City'] == 'Irvine'].head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
635,Irvine,California,34.6,125411.0,131516.0,256927,5532.0,116366.0,2.73,CA,American Indian and Alaska Native,799
1040,Irvine,California,34.6,125411.0,131516.0,256927,5532.0,116366.0,2.73,CA,Black or African-American,8670
1290,Irvine,California,34.6,125411.0,131516.0,256927,5532.0,116366.0,2.73,CA,Hispanic or Latino,19266
2295,Irvine,California,34.6,125411.0,131516.0,256927,5532.0,116366.0,2.73,CA,White,127358
2484,Irvine,California,34.6,125411.0,131516.0,256927,5532.0,116366.0,2.73,CA,Asian,126499


In [15]:
# City data exploration task 2
#Check to see if population adds up to total
#and check of any of the category is > Total Population
for _, row in city_df.iterrows():
    if pd.notnull(row['Male Population']):
        assert row['Male Population'] + row['Female Population'] == row['Total Population']
    if pd.notnull(row['Number of Veterans']):
        assert row['Number of Veterans'] <= row['Total Population']
    if pd.notnull(row['Foreign-born']):
        assert row['Foreign-born'] <= row['Total Population']
    if pd.notnull(row['Count']):
        assert row['Count'] <= row['Total Population']
    for numbered_column in ['Male Population', 'Female Population', 'Total Population', 'Number of Veterans', 'Foreign-born', 'Count']:
        assert pd.isnull(row[numbered_column]) or row[numbered_column] > 0


#looks like the data exploration task #2 passed without any issue.


In [16]:
#City data exploration task 3
#Check to see if race adds up to total
sub_df = city_df[['City', 'State', 'Total Population', 'Count']]
grouped = sub_df.groupby(['City', 'State', 'Total Population']).sum().reset_index()
grouped.head()

,City,State,Total Population,Count
0,Abilene,Texas,125876,147900
1,Akron,Ohio,197553,210305
2,Alafaya,Florida,85264,115476
3,Alameda,California,78614,89174
4,Albany,Georgia,71109,73478


In [17]:
#Let's look at if Count always sums up to >= Total Population

for _, row in grouped.iterrows():
    if pd.notnull(row['Total Population']) and row['Count'] < row['Total Population']:
        print(row['City'], row['State'], row['Total Population'], row['Count'])


Bayamón Puerto Rico 170259 169155
Caguas Puerto Rico 77008 76973
Mayagüez Puerto Rico 66581 65756
New Bedford Massachusetts 94959 93321
Ponce Puerto Rico 121583 120705
San Juan Puerto Rico 342237 342042
South Jordan Utah 66639 66205


Looks like the places with discrepancy is mostly in places lacked diversity.  

Let's look at the state level of data

In [18]:
sub_df = city_df[['State Code', 'Male Population', 'Female Population', 'Total Population', 'Foreign-born']]
state_df = sub_df.groupby(['State Code']).sum().reset_index()
print(state_df.shape)
state_df.sort_values(by=['Total Population'], ascending=False).head() 

(49, 5)


,State Code,Male Population,Female Population,Total Population,Foreign-born
4,CA,61055672.0,62388681.0,123444353,37059662.0
44,TX,34862194.0,35691659.0,70553853,14498054.0
34,NY,23422799.0,25579256.0,49002055,17186873.0
9,FL,15461937.0,16626425.0,32306132,7845566.0
14,IL,10943864.0,11570526.0,22514390,4632600.0


Looks like there isn't much we need to clean on the demographic data.

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model


Since this data project is based around the immigration data, we will use the immigration data as teh center of star schema (our fact table).
We will be calling this table as fact_immigration:
`
CREATE TABLE fact_immigration
(
    immigration_id int not null identity(0,1) primary key,
    cicid decimal not null,
    i94yr decimal not null,
    i94mon decimal not null,
    i94cit decimal,
    i94res decimal,
    i94port char(3),
    arrdate decimal,
    i94mode decimal,
    i94addr char(3),
    depdate decimal,
    i94bir decimal,
    i94visa decimal,
    count decimal,
    dtadfile varchar,
    visapost char(3),
    occup char(3),
    entdepa char(1),
    entdepd char(1),
    entdepu char(1),
    matflag char(1),
    biryear decimal,
    dtaddto varchar,
    gender char(1),
    insnum varchar,
    airline char(3),
    admnum varchar, 
    fltno varchar,
    visatype char(3)
    );
`



Here are the dimension tables:

dim_country
`
CREATE TABLE dim_country(
    code integer not null constraint dim_country_pkey primary key,
    name varchar not null
)
`

dim_arrival_mode

`
CREATE TABLE dim_arrival_mode(
    code integer,
    mode char(12)
)`

dim_port

`
CREATE TABLE dim_port(
    code char(3),
    name varchar
)`


dim_address

`
CREATE TABLE dim_address(
    code char(2),
    name varchar
)`

dim_visa_type

`
CREATE TABLE dim_visa_type(
    code integer,
    visa_type char(8)
)
`

dim_date

`
CREATE TABLE dim_date(
    code integer not null constraint dim_date_pkey primary key,
    year integer not null,
    month integer not null,
    day integer not null,
    day_of_week integer not null,
    ymd_dash char(10) not null,
    ymd_nodash char(8) not null,
    mdy_nodash char(8) not null
)`

dim_city

`
CREATE TABLE dim_city(
    city varchar,
    state varchar,
    median_age numeric,
    male_pop integer,
    female_pop integer, 
    total_pop integer,
    num_vets integer,
    foreign_born integer,
    avg_household_size DECIMAL,
    state_code char(2),
    race varchar,
    count integer
)
`

dim_state

`
CREATE TABLE dim_state(
    state_code char(2) constraint dim_state_pkey primary key,
    male_pop integer,
    female_pop integer,
    total_pop integer,
    foreign_born integer
)`


#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model



#### 1. fact_immigraiton

This one is by far the most complicated process since we would need to import large file sets.

We need to import each of the 12 files one at a time into memory (dataframe) first before we can insert into our redshift table.  After each insert, we would need to delete the dataframe to free up memory before proceeding to the next file.  

In addition, we found out the month of June actually contains extra columns, so we need to add code to clean up those unnecessary info before we can process the result.

#### 2. dim_arrival_mode and dim_visa_type

These two tables are more of a small lookup table.  We are just going to hardcode the insert.

#### 3. dim_address, dim_port, and dim_country

We are going to parse the value from the file and insert the result into the table

#### 4. dim_city and dim_state

We are going to insert directly from the dataframe we created at exploration stage and insert those into the refshift directly.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [39]:
#We are making db connection first
config = configparser.ConfigParser()
config.read('dbaccess.cfg')
connection_string = "host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values())
engine_connection_string='postgres://{}:{}@{}:{}/{}'.format(config['CLUSTER']['DB_USER'], config['CLUSTER']['DB_PASSWORD'], config['JDBC']['JDBCURL'], config['CLUSTER']['DB_PORT'], config['CLUSTER']['DB_NAME'])               
engine = create_engine(engine_connection_string)  # needed for DataFrame.to_sql

conn = psycopg2.connect(connection_string)
cur = conn.cursor()
conn.set_session(autocommit=True)

In [20]:
# Write code here
# We are going to clear the table if they exist first
sql_drop_dim_visa_type = 'DROP TABLE IF EXISTS dim_visa_type;'
cur.execute(sql_drop_dim_visa_type)

sql_drop_dim_state = 'DROP TABLE IF EXISTS dim_state;'
cur.execute(sql_drop_dim_state)

sql_drop_dim_country = 'DROP TABLE IF EXISTS dim_country;'
cur.execute(sql_drop_dim_country)

sql_drop_dim_city = 'DROP TABLE IF EXISTS dim_city;'
cur.execute(sql_drop_dim_city)

sql_drop_dim_date = 'DROP TABLE IF EXISTS dim_date;'
cur.execute(sql_drop_dim_date)

sql_drop_dim_address = 'DROP TABLE IF EXISTS dim_address;'
cur.execute(sql_drop_dim_address)

sql_drop_dim_port = 'DROP TABLE IF EXISTS dim_port;'
cur.execute(sql_drop_dim_port)

sql_drop_dim_arrival_mode = 'DROP TABLE IF EXISTS dim_arrival_mode;'
cur.execute(sql_drop_dim_arrival_mode)

#We are handling the fact_immigration in a python file
#sql_drop_fact_immigration = 'DROP TABLE IF EXISTS fact_immigration;'
#cur.execute(sql_drop_fact_immigration)


In [21]:
#create all tables
sql_create_dim_visa_type = '''CREATE TABLE dim_visa_type(
                                code integer,
                                visa_type char(8)
                            )'''
cur.execute(sql_create_dim_visa_type)

sql_create_dim_state = '''CREATE TABLE dim_state(
    state_code char(2) constraint dim_state_pkey primary key,
    male_pop decimal,
    female_pop decimal,
    total_pop decimal,
    foreign_born decimal
)'''
cur.execute(sql_create_dim_state)

sql_create_dim_country = '''CREATE TABLE dim_country(
    code integer not null constraint dim_country_pkey primary key,
    name varchar not null
)'''
cur.execute(sql_create_dim_country)

sql_create_dim_city = '''CREATE TABLE dim_city(
    city varchar,
    state varchar,
    median_age numeric,
    male_pop decimal,
    female_pop decimal, 
    total_pop decimal,
    num_vets integer,
    foreign_born decimal,
    avg_household_size DECIMAL,
    state_code char(2),
    race varchar,
    count integer
)'''
cur.execute(sql_create_dim_city)

sql_create_dim_date = '''CREATE TABLE dim_date(
    code integer not null constraint dim_date_pkey primary key,
    year integer not null,
    month integer not null,
    day integer not null,
    day_of_week integer not null,
    ymd_dash char(10) not null,
    ymd_nodash char(8) not null,
    mdy_nodash char(8) not null
)'''
cur.execute(sql_create_dim_date)

sql_create_dim_address = '''CREATE TABLE dim_address(
    code char(2),
    name varchar
)'''
cur.execute(sql_create_dim_address)

sql_create_dim_port = '''CREATE TABLE dim_port(
    code char(3),
    name varchar
)'''
cur.execute(sql_create_dim_port)

sql_create_dim_arrival_mode = '''CREATE TABLE dim_arrival_mode(
    code integer,
    mode char(12)
)'''
cur.execute(sql_create_dim_arrival_mode)

#creating of fact_immigration is being handled outside of the notebook
'''
sql_create_fact_immigration = CREATE TABLE fact_immigration(
    immigration_id int not null identity(0,1) primary key,
    cicid decimal not null,
    i94yr decimal not null,
    i94mon decimal not null,
    i94cit decimal,
    i94res decimal,
    i94port char(3),
    arrdate decimal,
    i94mode decimal,
    i94addr char(3),
    depdate decimal,
    i94bir decimal,
    i94visa decimal,
    count decimal,
    dtadfile varchar,
    visapost char(3),
    occup char(3),
    entdepa char(1),
    entdepd char(1),
    entdepu char(1),
    matflag char(1),
    biryear decimal,
    dtaddto varchar,
    gender char(1),
    insnum varchar,
    airline char(3),
    admnum varchar, 
    fltno varchar,
    visatype char(3)
    );
cur.execute(sql_create_fact_immigration)'''


'\nsql_create_fact_immigration = CREATE TABLE fact_immigration(\n    immigration_id int not null identity(0,1) primary key,\n    cicid decimal not null,\n    i94yr decimal not null,\n    i94mon decimal not null,\n    i94cit decimal,\n    i94res decimal,\n    i94port char(3),\n    arrdate decimal,\n    i94mode decimal,\n    i94addr char(3),\n    depdate decimal,\n    i94bir decimal,\n    i94visa decimal,\n    count decimal,\n    dtadfile varchar,\n    visapost char(3),\n    occup char(3),\n    entdepa char(1),\n    entdepd char(1),\n    entdepu char(1),\n    matflag char(1),\n    biryear decimal,\n    dtaddto varchar,\n    gender char(1),\n    insnum varchar,\n    airline char(3),\n    admnum varchar, \n    fltno varchar,\n    visatype char(3)\n    );\ncur.execute(sql_create_fact_immigration)'

Once we have create the table, we can start the data insert process

In [22]:
sql_insert_arrival_mode = """INSERT INTO dim_arrival_mode(code, mode)
SELECT 1, 'Air'
UNION
SELECT 2, 'Sea'
UNION
SELECT 3, 'Land'
UNION
SELECT 9, 'Not Reported'

"""
cur.execute(sql_insert_arrival_mode)



In [23]:
sql_insert_visa_type = """INSERT INTO dim_visa_type(code, visa_type)
SELECT 1, 'Business'
UNION
SELECT 2, 'Pleasure'
UNION
SELECT 3, 'Student'

"""
cur.execute(sql_insert_visa_type)



In [24]:
#sql_insert_address
with open(HEADER_FILE) as f:
    header_files = f.readlines()


In [25]:
comments = [line for line in header_files if line.startswith('/*') and line.endswith('*/\n')]
comment_pattern = re.compile(r'^/\*\s+(?P<code>.+?)\s+-\s+(?P<description>.+)\s+\*/$')
matches = [comment_pattern.match(cl) for cl in comments]

if not all(m is not None for m in matches):
    for i, m in enumerate(matches):
        if m is None:
            print(i)


matches = [comment_pattern.match(cl) for cl in comments]
if not all(m is not None for m in matches):
    for i, m in enumerate(matches):
        if m is None:
            print(i)
for m in matches:
    print('code:{}, description:{}'.format(m.group("code"), m.group('description')))


code:I94YR, description:4 digit year
code:I94MON, description:Numeric month
code:I94CIT & I94RES, description:This format shows all the valid and invalid codes for processing
code:I94PORT, description:This format shows all the valid and invalid codes for processing
code:I94MODE, description:There are missing values as well as not reported (9)
code:I94BIR, description:Age of Respondent in Years
code:COUNT, description:Used for summary statistics
code:DTADFILE, description:Character Date Field - Date added to I-94 Files - CIC does not use
code:VISAPOST, description:Department of State where where Visa was issued - CIC does not use
code:OCCUP, description:Occupation that will be performed in U.S. - CIC does not use
code:ENTDEPA, description:Arrival Flag - admitted or paroled into the U.S. - CIC does not use
code:ENTDEPD, description:Departure Flag - Departed, lost I-94 or is deceased - CIC does not use
code:ENTDEPU, description:Update Flag - Either apprehended, overstayed, adjusted to per

In [26]:
#Time to parse out address
address_lines = header_files[981:1036]

pattern = re.compile(r"^\s*'(?P<code>..)'\s*=\s*'(?P<name>.+)'.*$")
matches = [pattern.match(line) for line in address_lines]

address_codes = {match.group('code'): match.group('name') for match in matches}
assert len(address_codes) == len(address_lines)


In [27]:
sql_insert_address_template = 'INSERT INTO dim_address (code, name) VALUES (%s, %s);'

for item in sorted(address_codes.items()):
    cur.execute(sql_insert_address_template, item)


In [28]:
#Time to parse out port
port_lines = header_files[302:962]
pattern = re.compile(r"^\s*'(?P<code>...?)'\s*=\s*'(?P<name>.+)'.*$")
matches = [pattern.match(line) for line in port_lines]
port_codes = {match.group('code'): match.group('name').strip() for match in matches}
assert len(port_codes) == len(port_lines)


In [29]:
sql_insert_port_template = 'INSERT INTO dim_port (code, name) VALUES (%s, %s);'

for item in sorted(port_codes.items()):
    cur.execute(sql_insert_port_template, item)


In [30]:
#Time to parse out countries
country_lines = header_files[9:298]
pattern = re.compile(r"^\s*(?P<code>\d+)\s*=\s*'(?P<country>.+)'.*$")
matches = [pattern.match(line) for line in country_lines]
country_codes = {int(match.group('code')): match.group('country') for match in matches}
assert len(country_lines) == len(country_codes)

In [31]:
sql_insert_country_template = 'INSERT INTO dim_country (code, name) VALUES (%s, %s) '


for item in country_codes.items():
    cur.execute(sql_insert_country_template, item)


#### dim_date

Populating dim_date is different from other tables.  Since we have birth yeear from 1990 and the data was released on 2016, we will populate every day from 1/1/1900 to 12/31/2016

In [32]:
sql_insert_date_template = 'INSERT INTO dim_date(code, year, month, day, day_of_week, ymd_dash, ymd_nodash, mdy_nodash) values (%s, %s, %s, %s, %s, %s, %s, %s);'


dt = datetime(2016,1,1)
end_dt = datetime(2016, 12, 31)
one_day = timedelta(days=1)
code = 20454

while dt <= end_dt:
    cur.execute(sql_insert_date_template, 
               [code, dt.year, dt.month, dt.day, dt.weekday(), dt.strftime('%Y-%m-%d'),
                dt.strftime('%Y%m%d'), dt.strftime('%d%m%Y')]
               )
    dt = dt + one_day
    code += 1
    

dt = datetime(2015, 12, 31)
end_dt = datetime(1900, 1, 1)
code = 20453

while dt >= end_dt:
    cur.execute(sql_insert_date_template, 
               [code, dt.year, dt.month, dt.day, dt.weekday(), dt.strftime('%Y-%m-%d'),
                dt.strftime('%Y%m%d'), dt.strftime('%d%m%Y')]
               )
    dt = dt - one_day
    code -=1

In [33]:
#Inserting dim_city
sql_insert_city_template = '''INSERT INTO dim_city (city, state, median_age, male_pop, female_pop, total_pop
, num_vets, foreign_born, avg_household_size, state_code, race, count)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''

# execute queries
for _, row in city_df.iterrows():
    cur.execute(sql_insert_city_template, [v if pd.notna(v) else None for v in row])


In [35]:
#insert dim_state
sql_insert_state_template = '''INSERT INTO dim_state
(state_code, male_pop, female_pop, total_pop, foreign_born)
VALUES (%s, %s, %s, %s, %s)'''

# execute queries
for _, row in state_df.iterrows():
    cur.execute(sql_insert_state_template, row)


#### Handling fact_immigration

Because fact_immigration has huge amount of data, we want to handle the insert outside of the jupyter notebook as we do not wish to crash our internet browser.

I will be running load_fact_immgration_data.py in my local machine.

The approach that I used in the load_immigraiton_data.py is fairly straight forward.  Just read the sas data into dataframe, push the dataframe through Amazon's S3 through CSV file format.  Then we utilized the COPY command from RedShift.  This approach cuts the import time down dramatically.

python load_immigration_data.py

Necessary libraries are pandas, pscopg2, tqdm. These can be installed via

pip install -r requirements.txt

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

The first data quality check is actually just to look at how many rows we have by month

In [40]:
# Perform quality checks here
#using the following sql statement:
sql_data_quality_check_template = '''select count(distinct immigration_id), i94mon
from fact_immigration
group by i94mon
order by i94mon;'''
dq_check_df = pd.read_sql_query(sql_data_quality_check_template, engine)
dq_check_df


,count,i94mon
0,500000,1.0
1,500000,2.0
2,500000,3.0
3,500000,4.0
4,500000,5.0
5,500000,6.0
6,500000,7.0
7,500000,8.0
8,500000,9.0
9,500000,10.0


The second data quality check is to make sure we only have data from year 2016. 

As we can see from the result, we do not have any data outside of 2016.

In [41]:
sql_data_quality_check_template = 'select count(1) as Total from fact_immigration where i94yr is null or i94yr <> 2016;'
dq_check_df = pd.read_sql_query(sql_data_quality_check_template, engine)
dq_check_df

,total
0,0


The third data quality check is to examin into the cicid column.  From the first glance, this field looks like the primary key for the immigration data.  Let's explore it and see if that's the case.

In [42]:
sql_data_quality_check_template = 'select cicid, count(cicid) from fact_immigration group by cicid order by count desc;'
dq_check_df = pd.read_sql_query(sql_data_quality_check_template, engine)
dq_check_df

,cicid,count
0,32175.0,12
1,34883.0,12
2,33202.0,12
3,34926.0,12
4,34934.0,12
5,35286.0,12
6,35041.0,12
7,35765.0,12
8,35829.0,12
9,36163.0,12


It looks like there are few cicid's that appear multiple times.  Let's examin some of those rows see if we can find out more.

After inspecting the samples (see below), we know that these are not duplicate records and we made a good decision for not using cicid as our primary key.

In [43]:
sql_data_quality_check_template = '''select * from fact_immigration
where cicid in (5454856, 3334634, 4087143, 395680)
order by cicid;'''
dq_check_df = pd.read_sql_query(sql_data_quality_check_template, engine)
dq_check_df

,immigration_id,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,...,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,421527,395680.0,2016.0,1.0,516.0,516.0,FTL,20455.0,1.0,US,20463.0,35.0,2.0,1.0,...,PTS,,G,O,,M,1981.0,07012016,M,,BW,84246707530.0,00480,B2
1,1360193,395680.0,2016.0,2.0,687.0,687.0,MIA,20486.0,1.0,CO,20498.0,47.0,2.0,1.0,...,BNS,,G,O,,M,1969.0,08012016,M,,AA,87088113930.0,00900,B2
2,426916,395680.0,2016.0,6.0,690.0,690.0,HOU,20607.0,1.0,FL,20619.0,11.0,2.0,1.0,...,,,G,O,,M,2005.0,08302016,F,,UA,61347834233.0,00846,WT
3,1402578,395680.0,2016.0,3.0,438.0,438.0,LOS,20516.0,1.0,LA,20521.0,29.0,2.0,1.0,...,,,G,O,,M,1987.0,05312016,M,,VA,53739138933.0,00001,WT
4,2363804,395680.0,2016.0,4.0,582.0,582.0,SNA,20546.0,1.0,TX,20577.0,45.0,1.0,1.0,...,SNJ,,G,O,,M,1971.0,10012016,M,,4O,92556376030.0,02950,B1
5,3457726,395680.0,2016.0,7.0,213.0,296.0,NYC,20637.0,1.0,NY,20650.0,50.0,2.0,1.0,...,ABD,,G,O,,M,1966.0,01012017,M,,EK,15670443340.0,00201,B2
6,2409143,395680.0,2016.0,5.0,691.0,691.0,DAL,20576.0,1.0,TX,20581.0,39.0,2.0,1.0,...,BGT,,G,O,,M,1977.0,11012016,M,,AV,95166856730.0,00440,B2
7,5471178,395680.0,2016.0,10.0,689.0,689.0,MIA,20729.0,1.0,FL,20735.0,27.0,2.0,1.0,...,RDJ,,G,O,,M,1989.0,04012017,F,,AA,19371516140.0,00234,B2
8,4464564,395680.0,2016.0,8.0,528.0,528.0,MIA,20668.0,1.0,WI,20692.0,25.0,2.0,1.0,...,,,G,O,,M,1991.0,02012017,F,182766,AA,4637606585.0,1030,B2
9,6433302,395680.0,2016.0,11.0,213.0,213.0,SFR,20761.0,1.0,CA,20823.0,64.0,2.0,1.0,...,MDR,,G,O,,M,1952.0,05022017,F,25442,BA,13013813185.0,287,B2


As we found out in data exploration stage, there are some of the birth year we need to fix/exclude.  So we will run the below sql statement

In [44]:
sql_data_quality_fix = '''UPDATE fact_immigration
SET i94bir = NULL
WHERE i94bir < 0 or i94bir > 120;'''

cur.execute(sql_data_quality_fix)
sql_data_quality_fix = '''UPDATE fact_immigration
SET biryear = NULL
WHERE biryear < 1900 or biryear > 2016;'''

cur.execute(sql_data_quality_fix)


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

##### dim_visa_type
This table contains the type visa immigration uses:

- code: 1, 2, 3
- visa_tyope: Business, Pleasure, or Student, respectively

##### dim_state
This table contains the aggregated statistics from dim_city by state

- state_code: two-letter code for state
- male_pop: number of men in the state
- female_pop: number of women in the state
- total_pop: number of people in the state
- foreign_born: number of foreign-born people in the state

##### dim_country
This table stores the country name and code from our immigration data

- code: a numbered code
- name: usually a name of a country. There are many that start with INVALID: as well as several different No Country Code([code]) values

##### dim_city
This table stores the statistics on cities in US

- city: city's name
- state: state city is in
- median_age: median age of city
- male_pop: number of men in the city
- female_pop: number of women in the city
- total_pop: number of people in the city
- num_vets: number of veterans in the city
- foreign_born: number of foreign-born people in the city
- avg_household_size: average household size
- state_code: two-letter code for state
- race: White, Hispanic or Latino, Asian, Black or African-American, or American Indian and Alaska Native
- count: number of people of that race in the city

##### dim_date
This table stores the dates in different formats

- code: the CIC code for date where 20454 is 1/1/2016
- year: four-digit year
- month: month; 1-12
- day: day; 1-31
- day_of_week: 0 for Monday, 1 for Tuesday, ..., 7 for Sunday
- ymd_dash: date formatted as YYYY-MM-DD
- ymd_nodash: date formatted as YYYYMMDD
- mdy_noash: date formatted as MMDDYYYY

##### dim_address
This table stores the list of the states (usually) where immigrants list their address

- code: mostly two-letter codes for states. There's DC, GU (Guam), and 99 (All Other Codes) as well
- name: name of state, region, etc.

##### dim_port
This table stores the list of the ports from how the immigrants arrived

- code: a short code
- name: the name of the port; there are some No PORT Code ([code]) values too

##### dim_arrival_mode
This table stores the methopd in how those immigrants arrived. 

- code: 1, 2, 3, or 9
- mode: Air, Sea, Land, or Not reported, respectively

##### fact_immigration
This table is the fact table for our data capstone project

- immigration_id: primary key
- cicid: unique key within a month
- i94yr: 4 digit year, always 2016
- i94mon: numeric month, 1-12
- i94cit: immigrant's country of citizenship; foreign key to dim_country
- i94res: immigrant's country of residence outside US; foreign key to dim_country
- i94port: port of entry; foreign key to dim_port
- arrdate: arrival date of immigrant where 20454 == 1/1/2016
- i94mode: mode of arrival; foreign key to dim_arrival_mode
- i94addr: address (usually state) of immigrant in US; foreign key to dim_address
- depdate: departure date of immigrant where 20454 == 1/1/2016
- i94bir: immigrant's age in years
- i94visa: foreign key to dim_visa_type
- count: used for summary statistics; always 1 (for easy adding)
- dtadfile: dates in the format YYYYMMDD
- visapost: three-letter codes corresponding to where visa was issued
- occup: occupation in US of immigration. Mostly STU for student, also many OTH for other
- entdepa: one-letter arrival code
- entdepd: one-letter departure code
- entdepu: one-letter update code
- matflag: M if the arrival and departure records match
- biryear: four-digit year of birth
- dtaddto: MMDDYYYY date field for when the immigrant is admitted until
- gender: mostly M and F, but some X and U as well
- insnum: Immigration and Naturalization Services number; many re-used
- airline: Airline of entry for immigrant
- admnum: admission number; many re-used, but not as much as insnum
- fltno: flight number of immigrant
- visatype: short visa codes like WT, B2, WB, etc.

#### Step 5: Complete Project Write Up
##### * Clearly state the rationale for the choice of tools and technologies for the project.

Tools and technologies

For this project, I have decided to choose Python because it has a lot of useful libraries available to get the job done.  As an example, we use two different sql libraries so we can easily insert data into our RedShift datawarehouse (either with individual statement or from dataframe directly).

And by exploring data using Jupyter Notebook, we can examine data interactily.  

Also, python gives us an option to execute code via the console so we can be more process efficient when importing the large immigration files.

We are very lucky that the immigration data set is fairly clean and we do not need to spend a lot of time to clean or transform those data.

The only thing I changed is to replace NaN value for decimal into 0 and empty string for other NaN data.

We decided to use RedShift as our main database backend because we can easily scale up when we need to look at more data (the beauty of the cloud).

##### * Propose how often the data should be updated and why.

We should be importing the immigration data on monthly basis as we receive new data.  Due to the current unstable US administration, I am curious how that affect the movement of immigration if we can tie to the change upon activation of new immigration law.  

And as we import more immigration data, we will need to update the remaining dimension tables to cover the changes.



##### * Write a description of how you would approach the problem differently under the following scenarios:


##### * The data was increased by 100x.

If the data increase by 100x, we will want to utilize AirFlow as we can schedule and monitor the importing progress better.  Jupyter notebook as it's strength and weakness.  Process large amount of data is definitely something you don't want to do with the notebook.  We might also want to partition the fact table by date.

##### * The data populates a dashboard that must be updated on a daily basis by 7am every day.

We can use Airflow and utilize it's SLA feature to monitor and make sure data arrives according to the requirement.,


##### * The database needed to be accessed by 100+ people.

This will not be a problem since we currently putting data into the RedShift already.  We can easily scale up the process to support 100+ people.